# House Prices - Advanced Regression Techniques

Датасет взят из https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Результат сабмита = 0.14239

In [ ]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
#Загружаем данные из файла
df = pd.read_csv("../data/house_prices/train.csv", index_col=0)
df.head()

SalePrice - Цена продажи. Таргетная переменная, которую необходимо предсказать.

В остальных колонках содержится информация о доме(площадь лота, год постройки, тип дороги у дома и тд) или о сделке(месяц,год продажи,тип сделки)

# Подготовка данных

## Вещественные признаки

In [ ]:
# всего строк
df.shape

In [ ]:
# выделим числовые колонки и посмотрим их описание
numeric_columns = df.loc[:, df.dtypes != object].columns
df.describe()

В описании датасета для большинства случаев отсутствующее значение == отсутствию данного признака у обьекта(как например, отсутствует подвал, отсутствует земля перед домом и тд)

Для вещественных колонок заполним пропущенные значения нулями

In [ ]:
for col in numeric_columns:
    df[col] = df[col].fillna(0)

Проверим результат

In [ ]:
df.isna().sum()

Посмотрим корреляцию между признаками

In [ ]:
df[numeric_columns].corr()

Т.к признаков много,

для отображения N наиболее скоррелированных признаков можно использовать следующий код

In [ ]:
def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i + 1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop


def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]


print("Top Absolute Correlations")
print(get_top_abs_correlations(df[numeric_columns], 5))  #выведем топ 5 коррелирующих пар

Уберем признак GarageCars,нтуитивно кажется что его можно вычислить из GarageArea


По той же причине уберем TotRmsAbvGrd

Для колонок TotalBsmtSF и 1stFlrSF можно поспорить,что это не всегда одно и то же, но в силу высокой корреляции лучше всё же оставить что то одно,
Оставим площадь 1-го этажа

In [ ]:
df = df.drop(['GarageCars', 'TotRmsAbvGrd', 'TotalBsmtSF'], axis=1)
numeric_columns = numeric_columns.drop(['GarageCars', 'TotRmsAbvGrd', 'TotalBsmtSF'])

Теперь когда мы посмотрели корреляцию,можно убрать таргетную переменную из пула числовых колонок

In [ ]:
numeric_columns = numeric_columns.drop('SalePrice')

Далее посмотрим квазиконстантные признаки

In [ ]:
from sklearn.feature_selection import VarianceThreshold

cutter = VarianceThreshold(threshold=0.1)
cutter.fit(df[numeric_columns])
constant_cols = [x for x in numeric_columns if x not in cutter.get_feature_names_out()]

df[constant_cols]

In [ ]:
df['KitchenAbvGr'].value_counts()

In [ ]:
df['BsmtHalfBath'].value_counts()

Обе колонки больше похожи на категориальные, оставляем как есть.

## Не числовые признаки

In [ ]:
non_numeric_columns = df.loc[:, df.dtypes == object].columns
df.describe(include='object')

Некоторые колонки содержат пустые значения

In [ ]:
df[non_numeric_columns].isna().sum()

Напомним, в описании к датасету и колонкам указано, что NaN означает что этот признак у дома отсутствует(нет дороги, нет подвала и тд.)
Везде заменим отсутствующие значения на категорию 'None'

In [ ]:
for col in non_numeric_columns:
    df[col] = df[col].fillna('None')

In [ ]:
#Посмотрим данные ещё раз
df.describe(include='object')

# Обучение модели

In [ ]:
#Разделим данные на признаки и таргет
from sklearn.model_selection import train_test_split

X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Для колонок с количеством уникальных значений < 5 применим OneHotEncoder, для остальных - TargetEncoder

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from category_encoders.one_hot import OneHotEncoder

cols_for_ohe = [x for x in non_numeric_columns if X_train[x].nunique() < 5]
cols_for_mte = [x for x in non_numeric_columns if X_train[x].nunique() >= 5]

### Cохраним индексы этих колонок

cols_for_ohe_idx = [list(X_train.columns).index(col) for col in cols_for_ohe]
cols_for_mte_idx = [list(X_train.columns).index(col) for col in cols_for_mte]

t = [
    ('Numeric', StandardScaler(), numeric_columns),
    ('OneHotEncoder', OneHotEncoder(), cols_for_ohe_idx),
    ('MeanTargetEncoder', TargetEncoder(), cols_for_mte_idx)
]

col_transform = ColumnTransformer(transformers=t)

In [ ]:
#Подбираем альфа параметр для Ridge
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.linear_model import  Ridge
from sklearn.pipeline import Pipeline


pipe_ridge = Pipeline([
    ("column_transformer",
     col_transform),
    ('ridge', Ridge(max_iter=120000))])

alphas = np.linspace(0.01, 10, 30)

param_grid = {
    "ridge__alpha": alphas
}

search_ridge = GridSearchCV(pipe_ridge, param_grid, scoring='neg_root_mean_squared_error')
search_ridge.fit(X_train, y_train)

print(f"Best parameter (CV score={search_ridge.best_score_:.5f}):")
print(search_ridge.best_params_)

In [ ]:
#Фиксируем финальный результат на Ridge модели с лучшими параметрами.
pipe_ridge.set_params(ridge__alpha=search_ridge.best_params_['ridge__alpha'])

In [ ]:
cv_result_ridge = cross_validate(pipe_ridge, X_train, y_train, scoring='neg_root_mean_squared_error',
                                 return_train_score=True)

print(f'На трейне: {np.mean(cv_result_ridge["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result_ridge["test_score"]).round(3)}')

In [ ]:
pipe_ridge.fit(X_train, y_train)

In [ ]:
pipe_ridge.score(X_test, y_test)

То же самое но с LightGBM

In [ ]:
from lightgbm import LGBMRegressor

pipe_lg = Pipeline([
    ("column_transformer",
     col_transform),
    ('lg', LGBMRegressor(objective='regression',metric='rmse',random_state=42,verbose=-1))])

param_grid = {
    'lg__n_estimators': [50, 100, 200],
    'lg__max_depth': [2, 4, 6],
    'lg__min_split_gain': [2, 5, 10],
    'lg__min_child_samples': [1, 2, 4],
    'lg__learning_rate' : [0.01,0.001,0.1]
}

search_rf = GridSearchCV(pipe_lg, param_grid, scoring='neg_root_mean_squared_error')
search_rf.fit(X_train, y_train)

print(f"Best parameter (CV score={search_rf.best_score_:.5f}):")
print(search_rf.best_params_)

In [ ]:
pipe_lg.set_params(lg__n_estimators=search_rf.best_params_['lg__n_estimators'])
pipe_lg.set_params(lg__max_depth=search_rf.best_params_['lg__max_depth'])
pipe_lg.set_params(lg__min_split_gain=search_rf.best_params_['lg__min_split_gain'])
pipe_lg.set_params(lg__min_child_samples=search_rf.best_params_['lg__min_child_samples'])

In [ ]:
pipe_lg.fit(X_train, y_train)

In [ ]:
cv_result_lg = cross_validate(pipe_lg, X_train, y_train, scoring='neg_root_mean_squared_error',
                                 return_train_score=True)

print(f'На трейне: {np.mean(cv_result_lg["train_score"]).round(3)}')
print(f'На тесте: {np.mean(cv_result_lg["test_score"]).round(3)}')

# Предсказание на тесте и подготовка файла для сабмита

In [ ]:
test = pd.read_csv('../data/house_prices/test.csv')

In [ ]:
test.describe()

In [ ]:
test.isna().sum()

In [ ]:
test_filled = test.fillna(test.median(numeric_only=True))

In [ ]:
submission = pd.DataFrame()
submission['Id'] = test['Id']
submission['SalePrice'] = pipe_lg.predict(test_filled)
submission.to_csv('submission.csv', index=False)
submission.head()